In [1]:
import os
import sys
import gc
import math
import configparser
import numpy as np
import pandas as pd
import optuna
from tqdm import tqdm
import sklearn
import tensorflow as tf
from tensorflow import keras
from typing import Tuple

In [2]:
pd.set_option("use_inf_as_na", True)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)

In [3]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print(f"REPLICAS={strategy.num_replicas_in_sync}")

REPLICAS=1


In [4]:
KAGGLE_KERNEL_RUN_TYPE = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost')
IS_KAGGLE = KAGGLE_KERNEL_RUN_TYPE != 'Localhost'
INPUT = 'input'
DATA = INPUT
OUTPUT = 'output'
RESOURCE_DIR = '.'
if IS_KAGGLE:
    INPUT = '/kaggle/input'
    DATA = f'{INPUT}/shopee-product-matching'
    OUTPUT = '/kaggle/working'
    RESOURCE_DIR = f'{INPUT}/shopee-product-matching-lib/kaggle-shopee-product-matching-1.0'
    sys.path.append(f'{INPUT}/sgcharts-ml/src')
    sys.path.append(f"{INPUT}/sentence-transformers/sentence-transformers-1.0.4")
    sys.path.append(f'{RESOURCE_DIR}/src')
import mylib
import scml
from scml.nlp import strip_punctuation, to_ascii_str
scml.seed_everything()

In [5]:
MODEL = 'efficientnetb0'
CONF = configparser.ConfigParser()
CONF.read(f"{RESOURCE_DIR}/app.ini")
resolution = int(CONF[MODEL]["resolution"])
INPUT_SHAPE = (resolution, resolution, 3)
print(f"INPUT_SHAPE={INPUT_SHAPE}")

INPUT_SHAPE=(224, 224, 3)


In [6]:
train = pd.read_csv(f"{DATA}/train.csv", engine="c", low_memory=False)
train["target"] = mylib.target_label(train)
le = sklearn.preprocessing.LabelEncoder()
train["label_group"] = le.fit_transform(train['label_group'])
n_classes=len(le.classes_)
print(f"n_classes={n_classes}")
train.info()

n_classes=11014
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34250 entries, 0 to 34249
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   posting_id   34250 non-null  object
 1   image        34250 non-null  object
 2   image_phash  34250 non-null  object
 3   title        34250 non-null  object
 4   label_group  34250 non-null  int64 
 5   target       34250 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.6+ MB


In [7]:
train.head()

,posting_id,image,image_phash,title,label_group,target
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,666,"[train_129225211, train_2278313361]"
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DOUBLE FOAM TAPE",7572,"[train_3386243561, train_3423213080]"
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,6172,"[train_2288590299, train_3803689425]"
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Campur - Leher Kancing (DPT001-00) Batik karakter Alhadi,10509,"[train_2406599165, train_3342059966]"
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,9425,"[train_3369186413, train_921438619]"


In [8]:
def _data_gen(
    dataframe,
    directory,
    target_size,
    batch_size,
    color_mode="rgb",
    class_mode="raw",
    x_col="image",
    y_col="label_group"
):
    dtype = np.float32
    rescale = 1./255
    interpolation = "nearest"
    data_format = "channels_last"
    shuffle = True
    idg = keras.preprocessing.image.ImageDataGenerator(
        #shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True,
        rescale=rescale,
        data_format=data_format,
        dtype=dtype
    )
    g = idg.flow_from_dataframe(
        dataframe=dataframe,
        x_col=x_col,
        y_col=y_col,
        directory=directory,
        target_size=target_size,
        color_mode=color_mode,
        batch_size=batch_size,
        shuffle=shuffle,
        class_mode=class_mode,
        interpolation=interpolation,
    )
    while True:
        x, y = g.next()
        yield [x, y], y

In [9]:
def _model(
    pretrained,
    n_classes: int,
    lr: float,
    input_shape: Tuple[int, int, int],
    dtype=np.float32
):
    pretrained.trainable = False
    #kernel_initializer = keras.initializers.he_normal()
    #kernel_regularizer = keras.regularizers.l2(0.01)
    image_input = keras.layers.Input(shape=input_shape, name="image_input")
    label_input = keras.layers.Input(shape=(), name="label_input")
    x = pretrained(image_input)
    x = keras.layers.LayerNormalization()(x)
    x = keras.layers.Dense(pretrained.output_shape[1], activation="relu")(x)
    x = keras.layers.LayerNormalization(name="embedding_output")(x)
    x = mylib.ArcMarginProduct(
        n_classes=n_classes, 
        s=30, 
        m=0.5, 
        name='head/arc_margin', 
        dtype=dtype
    )([x, label_input])
    output = tf.keras.layers.Softmax(dtype=dtype)(x)
    model = tf.keras.models.Model(inputs = [image_input, label_input], outputs = [output])
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    loss = keras.losses.SparseCategoricalCrossentropy()
    sca = keras.metrics.SparseCategoricalAccuracy()
    model.compile(loss=loss, optimizer=optimizer, metrics=[sca])
    return model

In [10]:
pretrained = mylib.efficient_net(
    variant=MODEL,
    pooling="avg",
    directory=f"{RESOURCE_DIR}/pretrained/efficientnet",
)
model = _model(
    pretrained=pretrained,
    input_shape=INPUT_SHAPE,
    n_classes=n_classes,
    lr=1e-3,
)
model.summary(line_length=150)

Model: "model"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
image_input (InputLayer)                         [(None, 224, 224, 3)]            0                                                                   
______________________________________________________________________________________________________________________________________________________
efficientnetb0 (Functional)                      (None, 1280)                     4049571           image_input[0][0]                                 
______________________________________________________________________________________________________________________________________________________
layer_normalization (LayerNormalization)         (None, 1280)                  

In [11]:
def _callbacks(patience: int, directory: str):
    return [
        keras.callbacks.EarlyStopping(
            monitor="loss", patience=patience, verbose=1
        ),
        keras.callbacks.ModelCheckpoint(
            filepath=f"{directory}/model.h5",
            monitor="loss",
            save_best_only=True,
            verbose=1
        )
    ]

In [12]:
class MyObjective:
    def __init__(
        self,
        df,
        epochs: int,
        batch_size: int,
        patience: int,
        job_dir: str,
        lr: Tuple[float, float],
        n_classes: int,
        input_shape: Tuple[int, int, int],
    ):
        self.df = df
        self.epochs = epochs
        self.batch_size = batch_size
        self.patience = patience
        self.job_dir = job_dir
        self.lr = lr
        self.n_classes = n_classes
        self.input_shape = input_shape
        self.history: List[Dict[str, Union[str, int, float]]] = []

    def __call__(self, trial):
        hist = {
            "trial_id": trial.number,
            "learning_rate": trial.suggest_loguniform(
                "learning_rate", self.lr[0], self.lr[1]
            ),
        }
        train_gen = _data_gen(
            dataframe=self.df,
            directory=f"{DATA}/train_images",
            target_size=self.input_shape[:2],
            batch_size=self.batch_size,
        )
        pretrained = mylib.efficient_net(
            variant=MODEL,
            pooling="avg",
            directory=f"{RESOURCE_DIR}/pretrained/efficientnet",
        )
        model = _model(
            pretrained=pretrained,
            input_shape=self.input_shape,
            n_classes=self.n_classes,
            lr=hist["learning_rate"],
        )
        directory = f"{self.job_dir}/trial_{hist['trial_id']}"
        history = model.fit(
            train_gen,
            epochs=self.epochs,
            steps_per_epoch=len(self.df) / self.batch_size + 1,
            #validation_steps=len(vi) / self.batch_size + 1,
            #validation_data=val_gen,
            callbacks=_callbacks(self.patience, directory=directory),
            verbose=1
        )
        #y_pred = model.predict(x_val, batch_size=self.batch_size)
        #score = metrics.mean_squared_error(y_val, y_pred, squared=False)
        #print(repr(history.history))
        score = history.history["sparse_categorical_accuracy"][-1]
        #log.info(f"score={score:.4f}, fold={fold}, trial={hist['trial_id']}")
        print(f"score={score:.4f}, trial={hist['trial_id']}")
        del model
        gc.collect()
        hist["score_worst"] = score
        self.history.append(hist)
        return hist["score_worst"]

In [13]:
obj = MyObjective(
    df=train,
    epochs=40,
    batch_size=512 * strategy.num_replicas_in_sync,  # B3: OOM if batch size > 128 
    patience=2,
    job_dir=OUTPUT,
    lr=(1e-3, 1e-3),
    n_classes=n_classes,
    input_shape=INPUT_SHAPE,
)
study = optuna.create_study(direction="maximize")
study.optimize(obj, n_trials=1)

[I 2021-05-08 12:41:39,121] A new study created in memory with name: no-name-226bdd81-7521-4547-81d2-0f5d31d58c35


Found 34250 validated image filenames.
Epoch 1/40
67/67 [==============================] - 3237s 48s/step - loss: 24.5915 - sparse_categorical_accuracy: 0.0000e+00

Epoch 00001: loss improved from inf to 24.31868, saving model to output/trial_0/model.h5
Epoch 2/40
67/67 [==============================] - 2731s 40s/step - loss: 23.2401 - sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: loss improved from 24.31868 to 23.19616, saving model to output/trial_0/model.h5
Epoch 3/40
67/67 [==============================] - 3383s 50s/step - loss: 22.7323 - sparse_categorical_accuracy: 0.0000e+00

Epoch 00003: loss improved from 23.19616 to 22.55634, saving model to output/trial_0/model.h5
Epoch 4/40
67/67 [==============================] - 3351s 49s/step - loss: 21.3055 - sparse_categorical_accuracy: 0.0000e+00

Epoch 00004: loss improved from 22.55634 to 20.51913, saving model to output/trial_0/model.h5
Epoch 5/40
67/67 [==============================] - 2293s 34s/step - loss: 16.5985 - s

[I 2021-05-08 18:13:07,854] Trial 0 finished with value: 0.0 and parameters: {'learning_rate': 0.001}. Best is trial 0 with value: 0.0.


In [14]:
history = pd.DataFrame.from_records(obj.history)
history.sort_values("score_worst", ascending=False, inplace=True, ignore_index=True)
history.to_csv(f"{OUTPUT}/cv.csv", index=False)
history.head()

,trial_id,learning_rate,score_worst
0,0,0.001,0.0
